# **TUIA - Procesamiento de Imágenes y Visión por Computadora (IA52)**
# **Trabajo Práctico Final**
### **Ejercicio 4 - Evaluación**
<br>

### *Alumno: Miguel Mussi*
### *Año: 2024*

---------------------
## **Tabla de contenidos**
1.   [**Librerías**](#)
2.   [**Ajustes Iniciales**](#)
3.   [**Exportamos el modelo como tensor**](#)
4.   [**Verificamos si los Bounding Boxes estan dentro del límite de la imagen**](#)
        1.   [*Inferencia con CPU*](#)
        2.   [*Inferencia con GPU - Sin TensorRT*](#)
        3.   [*Inferencia con GPU - Con TensorRT*](#)
5.   [**Guardamos las imagenes con las predicciones**](#)
6.   [**Escritura del archivo envido.json**](#)

## 1. **Librerías**

In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import time
import json
import torch
import shutil
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [ ]:
drive.mount('/content/drive')
from google.colab import drive

## 2. **Ajustes Iniciales**

In [ ]:
# Nombre del alumno
student_name = 'miguel_mussi'

# Ruta al archivo de pesos
model_path = 'model/weights/best.pt'
model_path_tensor = 'model/weights/best.engine'

# Ruta al directorio que contiene las imagenes
imgs_dir = 'data/eval/images/val'

# Ruta al directorio de destino de las detecciones
base_dir = 'data/out'
dets_dir = os.path.join(base_dir, student_name)

# Reestablecimiento del directorio de destino (eliminacion)
if os.path.exists(dets_dir):
    shutil.rmtree(dets_dir)
os.makedirs(dets_dir)

## 3. **Exportamos el modelo como tensor**

In [ ]:
# Cargamos el modelo
model = YOLO(model_path)

# Lo exportamos como un tensor para utilizar tensorRT con la GPU
model.export(format='engine', imgsz=608, dynamic=True, verbose=True, batch=8, workspace=2, half=True)

WARNING  TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.2.60  Python-3.9.19 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 268 layers, 68,170,755 parameters, 0 gradients, 257.6 GFLOPs

PyTorch: starting from 'model\weights\best.pt' with input shape (8, 3, 608, 608) BCHW and output shape(s) (8, 53, 7581) (130.5 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success  10.9s, saved as 'model\weights\best.onnx' (260.1 MB)

TensorRT: starting export with TensorRT 10.2.0.post1...
TensorRT: input "images" with shape(-1, 3, -1, -1) DataType.FLOAT
TensorRT: output "output0" with shape(-1, 53, -1) DataType.FLOAT
TensorRT: building FP16 engine as model\weights\best.engine
TensorRT: export success  564.1s, saved as 'model\weights\best.engine' (133.8 MB)

Export complete (566.2s)
Results saved to D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\model\weights
Predict:         y

'model\\weights\\best.engine'

## 4. **Inferencia sobre las imágenes y cálculamos el envido**

### 4.1. *Inferencia con CPU*

In [ ]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
device = torch.device('cpu')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Iteramos sobre los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Comprobamos si la cantidad de cartas es válida para calcular el envido
    if len(cards) <= 1 or len(cards) > 3:
        print('La cantidad de cartas no permite calcular el envido\n')
    else:
        # Procesamos cada carta detectada
        for card in cards:
            name = cards.names[int(card.boxes.cls)]

            # Separamos el número y el palo de la carta
            if len(name) == 3:
                number = name[0] + name[1] # número de la carta
                type = name[2] # palo de la carta
            else:
                number = name[0] # número de la carta
                type = name[1] # palo de la carta

            # Comprobamos si la carta es inválida
            if number in ['9', '8', 'J']:
                invalid_cards += 1

            # Agregamos la carta al diccionario correspondiente
            if type in card_types:
                card_types[type].append(number)

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
        else:
            # Calculamos el envido basado en las cartas detectadas
            for key, value in card_types.items():
                if len(value) == 2:
                    type = key # Guardamos el palo en otra variable para luego

                    # Calculamos el envido
                    for number in value:
                        if number not in ['10', '11', '12']: # Ignoramos los 10, 11 y 12
                            envido += int(number)
                    if envido == 20:
                        envido = 0
                elif len(value) == 3:
                    type = key # Guardamos el palo en otra variable para luego

                    # Filtramos los valores más altos exceptuando los 10, 11 y 12
                    filtered_values = [x for x in value if x not in [10, 11, 12]]
                    largest_values = sorted(filtered_values, reverse=True)[:2]

                    # Calculamos el envido
                    for number in largest_values:
                        envido += int(number)
                    if envido == 20:
                        envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido != 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,       # Puntos de envido
            'figure': type          # Palo
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
print(f'Tiempo de ejecución: {execution_time:.2f} segundos')


image 1/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173711753_HDR.jpg: 608x480 1 11C, 573.0ms
La cantidad de cartas no permite calcular el envido

image 2/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173737533.jpg: 608x480 1 1C, 1 2B, 1 5C, 476.0ms
image 3/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173758691.jpg: 608x480 1 1C, 1 2B, 551.0ms
image 4/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173822184_HDR.jpg: 608x480 1 3B, 1 6B, 1 7E, 521.0ms
image 5/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173828513_HDR.jpg: 608x480 1 3B, 1 6B, 1 7E, 547.0ms
image 6/32 D:\Cosas de la escuela\Visin por Computa

### 4.2. *Inferencia con GPU - Sin TensorRT*

In [ ]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
device = torch.device('cuda:0')
model = YOLO(model_path).to(device)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Procesamos los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Comprobamos si la cantidad de cartas es válida para calcular el envido
    if len(cards) <= 1 or len(cards) > 3:
        print('La cantidad de cartas no permite calcular el envido\n')
    else:
        # Procesamos cada carta detectada
        for card in cards:
            name = cards.names[int(card.boxes.cls)]

            # Separamos el número y el palo de la carta
            if len(name) == 3:
                number = name[0] + name[1] # número de la carta
                type = name[2] # palo de la carta
            else:
                number = name[0] # número de la carta
                type = name[1] # palo de la carta

            # Comprobamos si la carta es inválida
            if number in ['9', '8', 'J']:
                invalid_cards += 1

            # Agregamos la carta al diccionario correspondiente
            if type in card_types:
                card_types[type].append(number)

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
        else:
            # Calculamos el envido basado en las cartas detectadas
            for key, value in card_types.items():
                if len(value) == 2:
                    type = key # Guardamos el palo en otra variable para luego

                    # Calculamos el envido
                    for number in value:
                        if number not in ['10', '11', '12']: # Ignoramos los 10, 11 y 12
                            envido += int(number)
                    if envido == 20:
                        envido = 0
                elif len(value) == 3:
                    type = key # Guardamos el palo en otra variable para luego

                    # Filtramos los valores más altos exceptuando los 10, 11 y 12
                    filtered_values = [x for x in value if x not in [10, 11, 12]]
                    largest_values = sorted(filtered_values, reverse=True)[:2]

                    # Calculamos el envido
                    for number in largest_values:
                        envido += int(number)
                    if envido == 20:
                        envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido != 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,
            'figure': type
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
print(f'Tiempo de ejecución: {execution_time:.2f} segundos')


image 1/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173711753_HDR.jpg: 608x480 1 11C, 104.1ms
La cantidad de cartas no permite calcular el envido

image 2/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173737533.jpg: 608x480 1 1C, 1 2B, 1 5C, 31.0ms
image 3/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173758691.jpg: 608x480 1 1C, 1 2B, 30.0ms
image 4/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173822184_HDR.jpg: 608x480 1 3B, 1 6B, 1 7E, 31.0ms
image 5/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173828513_HDR.jpg: 608x480 1 3B, 1 6B, 1 7E, 30.0ms
image 6/32 D:\Cosas de la escuela\Visin por Computadora

### 4.3. *Inferencia con GPU - Con TensorRT*

In [ ]:
# Marca de tiempo antes de la inferencia
start_time = time.time()

# Cargamos el modelo
model = YOLO(model_path_tensor, task='detect')

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Diccionario que guardará la información para el JSON
card_js_file = {}

# Procesamos los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Contador de cartas inválidas y envido
    invalid_cards = 0
    envido = 20

    # Diccionario para almacenar los tipos de cartas
    card_types = {'O': [], 'C': [], 'E': [], 'B': []}

    # Obtenemos las cartas detectadas
    cards = result.cpu()

    # Guardamos las detecciones en un archivo de texto en formato YOLOv5
    cards.save_txt(os.path.join(dets_dir, img_filename.replace('.jpg', '.txt')), save_conf=True)

    # Comprobamos si la cantidad de cartas es válida para calcular el envido
    if len(cards) <= 1 or len(cards) > 3:
        print('La cantidad de cartas no permite calcular el envido\n')
    else:
        # Procesamos cada carta detectada
        for card in cards:
            name = cards.names[int(card.boxes.cls)]

            # Separamos el número y el palo de la carta
            if len(name) == 3:
                number = name[0] + name[1] # número de la carta
                type = name[2] # palo de la carta
            else:
                number = name[0] # número de la carta
                type = name[1] # palo de la carta

            # Comprobamos si la carta es inválida
            if number in ['9', '8', 'J']:
                invalid_cards += 1

            # Agregamos la carta al diccionario correspondiente
            if type in card_types:
                card_types[type].append(number)

        # Si hay cartas inválidas, no se puede calcular el envido
        if invalid_cards > 0:
            print('No se puede calcular el envido por cartas inválidas\n')
        else:
            # Calculamos el envido basado en las cartas detectadas
            for key, value in card_types.items():
                if len(value) == 2:
                    type = key # Guardamos el palo en otra variable para luego

                    # Calculamos el envido
                    for number in value:
                        if number not in ['10', '11', '12']: # Ignoramos los 10, 11 y 12
                            envido += int(number)
                    if envido == 20:
                        envido = 0
                elif len(value) == 3:
                    type = key # Guardamos el palo en otra variable para luego

                    # Filtramos los valores más altos exceptuando los 10, 11 y 12
                    filtered_values = [x for x in value if x not in [10, 11, 12]]
                    largest_values = sorted(filtered_values, reverse=True)[:2]

                    # Calculamos el envido
                    for number in largest_values:
                        envido += int(number)
                    if envido == 20:
                        envido = 0

    # Almacenamos los resultados del envido en el diccionario para el JSON
    if envido != 20:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': envido,
            'figure': type
        }
    else:
        card_js_file[img_filename] = {
            'total_cards': len(cards),
            'cards': card_types,
            'points': 0,
            'figure': 'N/A'
        }

# Marca de tiempo después de la inferencia
end_time = time.time()

# Calculamos el tiempo de ejecución
execution_time = end_time - start_time

# Mostramos que dispositivo se esta utilizando y el tiempo de ejecución
if device.type == 'cuda': print('\nEl modelo está utilizando la GPU.')
else: print('\nEl modelo está utilizando la CPU.')
print(f'Tiempo de ejecución: {execution_time:.2f} segundos')

Loading model\weights\best.engine for TensorRT inference...

image 1/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173711753_HDR.jpg: 640x640 1 11C, 13.0ms
La cantidad de cartas no permite calcular el envido

image 2/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173737533.jpg: 640x640 1 1C, 1 2B, 1 5C, 12.0ms
image 3/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173758691.jpg: 640x640 1 1C, 1 2B, 12.0ms
image 4/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173822184_HDR.jpg: 640x640 1 3B, 1 6B, 1 7E, 12.0ms
image 5/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173828513_HDR.jpg: 640x640 1 3B, 1 6B, 1 7E, 12.

## 5. **Guardamos las imagenes con las predicciones**

In [ ]:
# Cargamos el modelo
model = YOLO(model_path_tensor)

# Ejecutamos la inferencia
results = model(imgs_dir, stream=True)

# Directorio donde van a ir las deteciones
detection_dir = f'{base_dir}/detections/'

# Creamos el directorio si no existe
os.makedirs(detection_dir, exist_ok=True)

# Iteramos sobre los resultados
for result in results:
    # Obtenemos el nombre del archivo de imagen
    img_filename = os.path.basename(result.path)

    # Guardamos la detección
    result.save(filename=f'{detection_dir}/{img_filename}')

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading model\weights\best.engine for TensorRT inference...

image 1/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173711753_HDR.jpg: 640x640 1 11C, 22.0ms
image 2/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173737533.jpg: 640x640 1 1C, 1 2B, 1 5C, 28.0ms
image 3/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173758691.jpg: 640x640 1 1C, 1 2B, 22.0ms
image 4/32 D:\Cosas de la escuela\Visin por Computadora\Trabajo Prctico Final\Actual\Ejercicio 4\data\eval\images\val\IMG_20240630_173822184_HDR.jpg: 640x640 1 3B, 1 6B, 1 7E, 23.0ms
image 5/32 D:\Cosas de la escuela\Visin por Computadora\Tra

## 6. **Escritura del archivo envido.json**

In [ ]:
with open(os.path.join(dets_dir, 'envido.json'), 'w') as jf:
    json.dump(card_js_file, jf, indent=4)